### Advent of Code 2021 - day 26 [Le Wagon edition]

This was a custom made challenge for our Le Wagon community. It was prepared by one of our teammates as a little thank you for our organizer Pil0u. The main task was to decipher some messages, the decryption process was not straight forward of course.

Disclaimer: This code does only handle the decryption. The scraping of the other messages and necessary data is still to be completed.

In [1]:
# Le Wagon - AOC placement of part1 and part2
babs1 = [31,60,36,57,32,33,54,104,38,27,38,33,49,38,52,52,36,45,35,35,23,31,12,26,21]
babs2 = [29,60,58,95,27,42,114,108,118,76,38,36,50,59,67,51,36,44,33,37,36,33,14,25,25]

babs_id = 326
message = "sfwa8C)u8epY)k8G8 hsAxsrsn8CVsu8)C8ehwkh)y8RuPo))p8arYx8lhsf8xw8G8euw8CP8esf)p8G8CYA8!sxPC8DwxCDPhai8lD)r8lx8mY8eskshrD8y)8euPo8CG8 DY8h)y8Cusfs8ELGWvKv8DPaC8RuPCwshr8h)y8YépPd8PrhsK lCPuYxx)r8sx)Dskw8DPaC8)-k8hwy8DPaC8CsR8)C8spAw8ussA8"


---
#### part1: cipher - product and identifier
- If the sender identifier is an even number, it is added to the product of the decryption key sequence.
- If the sender identifier is an odd number, it is subtracted from the product of the decryption key sequence.

In [2]:
def calculate_cipher(id, part2):    
    decryption = 1
    for value in part2:
        decryption *= value

    if id%2 == 0:
        decryption += id
    else:
        decryption -= id
        
    return decryption


#### part2: cipher - every third digit
Using the absolute value of the above calculation result, the cipher is the sequence of every third digit, starting from the left.


In [3]:
def take_every_third(cipher):
    cipher = str(abs(decryption))
    cipher = cipher[0:-1:3]
    return cipher


#### part3: cipher - convert to ternary
The resulting cipher is then turned into a ternary number and processed from right to left.

In [4]:
def ternary(n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, 3)
        nums.append(str(r))
    return ''.join(reversed(nums))


#### part4: message - shift message according to ternary cipher

For each trit:
- If the trit is a 1, shift the entire encoded message left by one character (the first character becomes the last).
- If the trit is a 2, shift the entire encoded message right by the trit's magnitude (characters chopped off the end get added back to the beginning).
- If the trit is a 0, reverse the message string.

For cipher 46, the ternary representation is 1201. Suppose our string is abcde:
abcde -(shift left by 1)> 
bcdea -(reverse)> 
aedcb -(shift right by 3^2 = 9)> e
dcba -(shift left by 1)> dcbae

In [5]:
def shift_message(cipher, message):
    
    for index, c in enumerate(cipher[::-1]):
        if c == '1':
            # shift left
            message = message[1:] + message[:1]
        elif c == '2':
            # shift right by the magnitude, e.g. 3^2=9
            m = 3**index % len(message)
            message = message[-m:] + message[:-m]
        elif c == '0':
            # reverse
            message = message[::-1]
            
    return message


---

#### part5: Rudolphabet

North Pole HQ uses a special alphabet called the **Rudolphabet**. It works almost like the normal alphabet, except that the letter order follows their first appearance in the popular song lyrics. This includes, separately, lower- and upper-case letters. All remaining letters, punctuation marks, special characters and the space character are ignored and stay at the end. The Rudolphabet repeats over and over; if you have to shift further than its length, simply append the missing characters again.


North Pole HQ previously used the **Jinglebet**, which turned the standard alphabet
- ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzÇàéè0123456789 .!?,;:'-#/\()
- DashingtrouewOplfdLyBbmkWAjJIMFTNGHcCEKPQRSUVXYZqvxzÇàéè0123456789 .!?,;:'-#/\()

In [6]:
def create_rudolphabet():
    with open('day26_rudolph.txt') as file:
        songtext = file.readlines()

    rudolphabet = []
    punctuation = " .!?,;:'-#/\()"

    for line in songtext:
        for x in line.strip():
            if x not in rudolphabet and x not in punctuation:
                rudolphabet.append(x)

    abc = "ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzÇàéè0123456789 .!?,;:'-#/\()"
    for x in abc:
        if x not in rudolphabet:
            rudolphabet.append(x)
    return rudolphabet


#### part6: cipher - get prime factors

In [7]:
import math

def primeFactorsSum(n):
    prime_sum = 0

    while n % 2 == 0:
        prime_sum += 2
        n = n / 2
        
    for i in range(3, int(math.sqrt(n)) + 1, 2):
        while n % i == 0:
            prime_sum += i
            n = n / i
    if n > 2:
        prime_sum += n
    
    prime_sum = int(prime_sum)
    if prime_sum == n:
        prime_sum += 5
        
    return prime_sum


#### part7: message - Caesar Cipher
Finally, the shifted message characters are decoded using a **Caesar cipher**:
- Get the prime factors of the original cipher (in decimal) and shift each character along the Rudolphabet* by their sum.
- If the cipher is a prime number (i.e. if its only prime factor is itself), add 5 to the value before you shift.
- Using the Jinglebet, for cipher 46, we shift by 23+2 = 25: D becomes A, a becomes j, s becomes J and so on.


In [8]:
def decrypt_caesar_cipher(message, shift, alphabet):
    alph_length = len(alphabet)
    shift = shift % alph_length
    decrypted = ''
    for x in message:
        # find position in alphabet and find new position in alphabet
        for i, a in enumerate(alphabet):
            if a == x:
                index = (i + shift) % alph_length
                break
        # add that new character to decrypted
        decrypted += alphabet[index]
    return decrypted


---

In [11]:
babs2 = [29,60,58,95,27,42,114,108,118,76,38,36,50,59,67,51,36,44,33,37,36,33,14,25,25]
babs_id = 326
message = "sfwa8C)u8epY)k8G8 hsAxsrsn8CVsu8)C8ehwkh)y8RuPo))p8arYx8lhsf8xw8G8euw8CP8esf)p8G8CYA8!sxPC8DwxCDPhai8lD)r8lx8mY8eskshrD8y)8euPo8CG8 DY8h)y8Cusfs8ELGWvKv8DPaC8RuPCwshr8h)y8YépPd8PrhsK lCPuYxx)r8sx)Dskw8DPaC8)-k8hwy8DPaC8CsR8)C8spAw8ussA8"

cipher = calculate_cipher(babs_id, babs2)

cipher = take_every_third(cipher)

cipher_ternary = ternary(int(cipher))

message = shift_message(cipher_ternary, message)

rudolphabet = create_rudolphabet()

caesar_cipher = primeFactorsSum(int(cipher))

decrypted_message = decrypt_caesar_cipher(message, caesar_cipher, rudolphabet)

print(decrypted_message)


Merci Pil0u for creating this AMAZING event for us! It kind of screwed up my cosy Christmas time, but I loved it and I am very much looking forward to next December! I would not have been able to get this far w/o this awesome community!
